In [19]:
pip install openrouteservice

Note: you may need to restart the kernel to use updated packages.


In [20]:
# Import packages
import requests
import pandas as pd
import openrouteservice as ors
import csv
from openrouteservice.exceptions import ApiError
import math

In [21]:
API_key = '5b3ce3597851110001cf62489f516e047c31bc1d8c0fcd09f3a4d368c7e48da0f15d1dd0b5182663'

In [22]:
client = ors.Client(key=API_key)

# Computing distance to Venoge Festival

In [23]:
#vaud_cities = pd.read_csv(repo_url+'vaud_cities.csv')

In [24]:
# Initialize the coordinate of the Venoge festival
coordonnees_venoge = [46.60396069250175, 6.538034286509177] #lat,long

In [25]:
# Convert the coordinates in E & N format to WGS

# Convert CH y/x to WGS lat
def CHtoWGSlat(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lat = (16.9023892 + (3.238272 * x_aux)) + \
            - (0.270978 * pow(y_aux, 2)) + \
            - (0.002528 * pow(x_aux, 2)) + \
            - (0.0447 * pow(y_aux, 2) * x_aux) + \
            - (0.0140 * pow(x_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lat = (lat * 100) / 36
    return lat

        # Convert CH y/x to WGS long
def CHtoWGSlng( y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lng = (2.6779094 + (4.728982 * y_aux) + \
        + (0.791484 * y_aux * x_aux) + \
        + (0.1306 * y_aux * pow(x_aux, 2))) + \
        - (0.0436 * pow(y_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lng = (lng * 100) / 36
    return lng

In [26]:
'''
vaud_cities['E_lv03'] = vaud_cities['E']-2000000
vaud_cities['N_lv03'] = vaud_cities['N']-1000000 

vaud_cities['lat'] = vaud_cities.apply(lambda x: CHtoWGSlat(x['E_lv03'], x['N_lv03']), axis=1)
vaud_cities['lng'] = vaud_cities.apply(lambda x: CHtoWGSlng(x['E_lv03'], x['N_lv03']), axis=1)
'''


"\nvaud_cities['E_lv03'] = vaud_cities['E']-2000000\nvaud_cities['N_lv03'] = vaud_cities['N']-1000000 \n\nvaud_cities['lat'] = vaud_cities.apply(lambda x: CHtoWGSlat(x['E_lv03'], x['N_lv03']), axis=1)\nvaud_cities['lng'] = vaud_cities.apply(lambda x: CHtoWGSlng(x['E_lv03'], x['N_lv03']), axis=1)\n"

In [27]:
#vaud_cities

In [28]:
# Add a new column for distance to Venoge
#vaud_cities['distance_to_venoge_km'] = None

In [29]:
# Calculate the distance to Venoge for one city
''' 
route = client.directions(
    profile='driving-car',
    format='geojson',
    coordinates=[[6.968953, 46.313884],[6.5367833784414175, 46.60314203414339]] #longitude, latitude
)
print(route)
print(route['features'][0]['properties']['segments'][0]['distance'])
'''

" \nroute = client.directions(\n    profile='driving-car',\n    format='geojson',\n    coordinates=[[6.968953, 46.313884],[6.5367833784414175, 46.60314203414339]] #longitude, latitude\n)\nprint(route)\nprint(route['features'][0]['properties']['segments'][0]['distance'])\n"

In [30]:
# Calculate distance to Venoge for each city
''' 
for index, row in vaud_cities.iterrows():
    city=f"{row['lng']},{row['lat']}"
    print(row['Commune'])
    try:
        route = client.directions(
            profile='driving-car',
            format='geojson',
            coordinates=[
                [coordonnees_venoge[1], coordonnees_venoge[0]],  # [lng, lat]
                [row['lng'], row['lat']]
            ]
        )

        distance_km = route['features'][0]['properties']['segments'][0]['distance'] / 1000
        vaud_cities.at[index, 'distance_to_venoge_km'] = round(distance_km, 2)

    except ApiError as e:
        print(f"API error for {row['Commune']}: {e}")
        vaud_cities.at[index, 'distance_to_venoge_km'] = None

    except Exception as e:
        print(f"Unexpected error for {row['Commune']}: {e}")
        vaud_cities.at[index, 'distance_to_venoge_km'] = None
        '''

' \nfor index, row in vaud_cities.iterrows():\n    city=f"{row[\'lng\']},{row[\'lat\']}"\n    print(row[\'Commune\'])\n    try:\n        route = client.directions(\n            profile=\'driving-car\',\n            format=\'geojson\',\n            coordinates=[\n                [coordonnees_venoge[1], coordonnees_venoge[0]],  # [lng, lat]\n                [row[\'lng\'], row[\'lat\']]\n            ]\n        )\n\n        distance_km = route[\'features\'][0][\'properties\'][\'segments\'][0][\'distance\'] / 1000\n        vaud_cities.at[index, \'distance_to_venoge_km\'] = round(distance_km, 2)\n\n    except ApiError as e:\n        print(f"API error for {row[\'Commune\']}: {e}")\n        vaud_cities.at[index, \'distance_to_venoge_km\'] = None\n\n    except Exception as e:\n        print(f"Unexpected error for {row[\'Commune\']}: {e}")\n        vaud_cities.at[index, \'distance_to_venoge_km\'] = None\n        '

In [31]:
# The data are ordered by distance to Venoge
'''
vaud_cities.sort_values(by='distance_to_venoge_km', ascending=True, inplace=True)
vaud_cities.reset_index(drop=True, inplace=True)
vaud_cities
'''

"\nvaud_cities.sort_values(by='distance_to_venoge_km', ascending=True, inplace=True)\nvaud_cities.reset_index(drop=True, inplace=True)\nvaud_cities\n"

In [32]:
# Remove rows with NaN values in 'distance_to_venoge_km'
'''
vaud_cities[vaud_cities['distance_to_venoge_km'].isna()]
vaud_cities.dropna(subset=['distance_to_venoge_km'], inplace=True)
'''

"\nvaud_cities[vaud_cities['distance_to_venoge_km'].isna()]\nvaud_cities.dropna(subset=['distance_to_venoge_km'], inplace=True)\n"

In [33]:
# Save the updated DataFrame to a new CSV file
#vaud_cities.to_csv('vaud_cities.csv', index=False)

# Importing data

In [34]:
# Load the CSV file
repo_url='https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'
vaud_cities = pd.read_csv(repo_url+'vaud_cities.csv')
wed_tickets = pd.read_csv(repo_url+'data/ticket_ME.csv')
thu_tickets = pd.read_csv(repo_url+'data/ticket_JE.csv')
fri_tickets = pd.read_csv(repo_url+'data/ticket_VE.csv')
sat_tickets = pd.read_csv(repo_url+'data/ticket_SA.csv')
sub_tickets = pd.read_csv(repo_url+'data/ticket_abonnement.csv')
all_tickets = pd.read_csv(repo_url+'data/ticket_all.csv')
cities_CH = pd.read_csv(repo_url+'data/cities_CH.csv', sep=';')
cities_CH.rename(columns={'Kantonskürzel':'Canton'},inplace=True)
npa_to_connect = pd.read_excel(repo_url+'excluded_npa.xlsx',header=None)
display(vaud_cities)

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,96 ans,97 ans,98 ans,99 ans,100+ ans,E_lv03,N_lv03,lat,lng,distance_to_venoge_km
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,2,1,0,1,0,531186.492,161298.778,46.599406,6.540556,0.74
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,3,1,1,3,0,530510.884,162919.512,46.613915,6.531495,2.72
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,0,0,0,0,0,533025.028,162344.417,46.608998,6.564397,3.51
3,Vufflens-la-Ville,1302,Vufflens-la-Ville,5503,VD,2530896.906,1158606.892,fr,1342,7,...,1,0,0,0,0,530896.906,158606.892,46.575163,6.537181,3.65
4,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,0,0,0,0,0,531837.469,164336.340,46.626796,6.548602,3.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,La Lécherette,1660,Château-d'Oex,5841,VD,2573141.894,1141175.827,fr,3656,36,...,4,0,1,1,0,573141.894,141175.827,46.421392,7.089270,79.09
532,Morcles,1892,Lavey-Morcles,5406,VD,2569963.493,1117103.969,fr,1014,11,...,1,0,0,0,0,569963.493,117103.969,46.204719,7.049478,79.70
533,Flendruz,1659,Château-d'Oex,5841,VD,2578662.257,1151472.083,fr,3656,36,...,4,0,1,1,0,578662.257,151472.083,46.514211,7.160603,80.11
534,La Lécherette,1660,Ormont-Dessous,5410,VD,2570433.864,1139757.044,fr,1244,8,...,1,0,0,0,0,570433.864,139757.044,46.408515,7.054135,83.75


In [35]:
wed_tickets.drop(columns=['Unnamed: 0'], inplace=True)
thu_tickets.drop(columns=['Unnamed: 0'], inplace=True)
fri_tickets.drop(columns=['Unnamed: 0'], inplace=True)
sat_tickets.drop(columns=['Unnamed: 0'], inplace=True)
sub_tickets.drop(columns=['Unnamed: 0'], inplace=True)
all_tickets.drop(columns=['Unnamed: 0'], inplace=True)
all_tickets.dropna(inplace=True)
wed_tickets.dropna(inplace=True)
thu_tickets.dropna(inplace=True)
fri_tickets.dropna(inplace=True)
sat_tickets.dropna(inplace=True)
sub_tickets.dropna(inplace=True)

In [36]:
all_tickets.isna().sum()

PLZ       0
day       0
ticket    0
dtype: int64

In [37]:
all_tickets.groupby(by='day')['ticket'].sum()

day
ABO      323
JE      9077
ME     12975
SA      7609
VE      6434
Name: ticket, dtype: int64

In [38]:
cities_CH=cities_CH.drop_duplicates(subset='PLZ', keep='first')

In [39]:
# Add canton to the data
all_tickets, wed_tickets, thu_tickets, fri_tickets, sat_tickets,sub_tickets = [
    df.merge(cities_CH[['PLZ', 'Canton']], how='left', on='PLZ') 
    for df in [all_tickets, wed_tickets, thu_tickets, fri_tickets, sat_tickets,sub_tickets]
]

In [40]:
all_tickets['ticket'].sum()

36418

In [41]:
# Verify that the sum of tickets throughout the different days corresponds with the number of all tickets
all_tickets['ticket'].sum() == wed_tickets['ticket'].sum() + thu_tickets['ticket'].sum() +  fri_tickets['ticket'].sum() +  sat_tickets['ticket'].sum() +  sub_tickets['ticket'].sum()

True

In [42]:
wed_tickets[wed_tickets['Canton'] == 'VD']['ticket'].sum() + thu_tickets[thu_tickets['Canton'] == 'VD']['ticket'].sum() + fri_tickets[fri_tickets['Canton'] == 'VD']['ticket'].sum() + sat_tickets[sat_tickets['Canton'] == 'VD']['ticket'].sum() + sub_tickets[sub_tickets['Canton'] == 'VD']['ticket'].sum() #== all_tickets[all_tickets['Canton'] == 'VD']['ticket'].sum()

26671

In [43]:
all_tickets.groupby(by=['Canton']).agg({'ticket': 'sum'}).reset_index().rename(columns={'ticket': 'Nb Tickets'}).sort_values(by='Nb Tickets', ascending=False).head(10)

,Canton,Nb Tickets
17,VD,26671
5,FR,3094
6,GE,2067
18,VS,1696
9,NE,1673
2,BE,624
7,JU,297
20,ZH,98
15,TI,28
0,AG,24


In [44]:
NPA_vaud = vaud_cities['NPA'].unique().tolist()
print(len(NPA_vaud))

334


In [45]:
# Filter the NPA located in Canton Vaud in the tickets df
NPA_vaud_with_demand = all_tickets[all_tickets['Canton'] == 'VD']['PLZ'].unique().tolist()

In [46]:
# Filter the df of cities in Canton Vaud to keep those with demand
vaud_cities = vaud_cities[vaud_cities['NPA'].isin(NPA_vaud_with_demand)]
vaud_cities

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,96 ans,97 ans,98 ans,99 ans,100+ ans,E_lv03,N_lv03,lat,lng,distance_to_venoge_km
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,2,1,0,1,0,531186.492,161298.778,46.599406,6.540556,0.74
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,3,1,1,3,0,530510.884,162919.512,46.613915,6.531495,2.72
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,0,0,0,0,0,533025.028,162344.417,46.608998,6.564397,3.51
3,Vufflens-la-Ville,1302,Vufflens-la-Ville,5503,VD,2530896.906,1158606.892,fr,1342,7,...,1,0,0,0,0,530896.906,158606.892,46.575163,6.537181,3.65
4,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,0,0,0,0,0,531837.469,164336.340,46.626796,6.548602,3.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,La Lécherette,1660,Château-d'Oex,5841,VD,2573141.894,1141175.827,fr,3656,36,...,4,0,1,1,0,573141.894,141175.827,46.421392,7.089270,79.09
532,Morcles,1892,Lavey-Morcles,5406,VD,2569963.493,1117103.969,fr,1014,11,...,1,0,0,0,0,569963.493,117103.969,46.204719,7.049478,79.70
533,Flendruz,1659,Château-d'Oex,5841,VD,2578662.257,1151472.083,fr,3656,36,...,4,0,1,1,0,578662.257,151472.083,46.514211,7.160603,80.11
534,La Lécherette,1660,Ormont-Dessous,5410,VD,2570433.864,1139757.044,fr,1244,8,...,1,0,0,0,0,570433.864,139757.044,46.408515,7.054135,83.75


In [47]:
cities_to_cluster = vaud_cities[vaud_cities[['NPA']].duplicated()].sort_values(by=['NPA','distance_to_venoge_km'])
cities_to_cluster

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,96 ans,97 ans,98 ans,99 ans,100+ ans,E_lv03,N_lv03,lat,lng,distance_to_venoge_km
128,Lausanne 26,1000,Lausanne,5586,VD,2543135.035,1156066.783,fr,145037,1311,...,90,61,50,35,52,543135.035,156066.783,46.553466,6.697131,18.89
141,Lausanne 25,1000,Lausanne,5586,VD,2542510.369,1158315.802,fr,145037,1311,...,90,61,50,35,52,542510.369,158315.802,46.573643,6.688705,20.26
51,Prilly,1008,Prilly,5589,VD,2535977.633,1154306.959,fr,12766,110,...,12,3,4,1,2,535977.633,154306.959,46.536990,6.604050,11.17
60,Prilly,1008,Renens (VD),5591,VD,2535633.311,1153352.766,fr,21568,175,...,10,6,5,10,5,535633.311,153352.766,46.528374,6.599695,12.13
44,Crissier,1023,Lausanne,5586,VD,2534771.115,1156568.319,fr,145037,1311,...,90,61,50,35,52,534771.115,156568.319,46.557215,6.588004,10.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,Frenières-sur-Bex,1880,Bex,5402,VD,2571040.657,1121183.948,fr,8806,79,...,6,2,0,1,3,571040.657,121183.948,46.241467,7.063181,78.92
529,Gryon,1882,Bex,5402,VD,2576398.397,1126123.468,fr,8806,79,...,6,2,0,1,3,576398.397,126123.468,46.286110,7.132393,78.67
518,Arveyes,1884,Ollon,5409,VD,2570658.534,1126302.457,fr,8222,52,...,3,1,2,2,3,570658.534,126302.457,46.287494,7.057905,71.51
524,Villars-sur-Ollon,1884,Ollon,5409,VD,2574459.520,1129195.462,fr,8222,52,...,3,1,2,2,3,574459.520,129195.462,46.313673,7.107067,75.10


In [48]:
vaud_cities.shape

(499, 115)

In [49]:
# Cluster cities with the same NPA, we keep the city closest to the Venoge festival
cities_to_serve = vaud_cities.drop_duplicates(subset='NPA', keep='first')
cities_to_serve

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,96 ans,97 ans,98 ans,99 ans,100+ ans,E_lv03,N_lv03,lat,lng,distance_to_venoge_km
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,2,1,0,1,0,531186.492,161298.778,46.599406,6.540556,0.74
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,3,1,1,3,0,530510.884,162919.512,46.613915,6.531495,2.72
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,0,0,0,0,0,533025.028,162344.417,46.608998,6.564397,3.51
3,Vufflens-la-Ville,1302,Vufflens-la-Ville,5503,VD,2530896.906,1158606.892,fr,1342,7,...,1,0,0,0,0,530896.906,158606.892,46.575163,6.537181,3.65
4,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,0,0,0,0,0,531837.469,164336.340,46.626796,6.548602,3.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,La Comballaz,1862,Ormont-Dessous,5410,VD,2572060.524,1136176.786,fr,1244,8,...,1,0,0,0,0,572060.524,136176.786,46.376379,7.075504,72.24
522,Rougemont,1659,Château-d'Oex,5841,VD,2580115.427,1147183.777,fr,3656,36,...,4,0,1,1,0,580115.427,147183.777,46.475680,7.179722,74.02
526,Leysin,1854,Leysin,5407,VD,2566911.342,1133415.264,fr,3685,23,...,1,1,0,0,1,566911.342,133415.264,46.351303,7.008778,75.77
527,Gryon,1882,Gryon,5405,VD,2573628.043,1126382.007,fr,1525,10,...,0,0,0,1,0,573628.043,126382.007,46.288333,7.096432,77.00


In [50]:
all_tickets.loc[all_tickets['PLZ']==1854,'ticket']=all_tickets.loc[all_tickets['PLZ']==1854,'ticket'] + all_tickets.loc[all_tickets['PLZ']==1865,'ticket']
all_tickets = all_tickets[all_tickets['PLZ'] != 1865]
wed_tickets.loc[wed_tickets['PLZ']==1854,'ticket']=wed_tickets.loc[wed_tickets['PLZ']==1854,'ticket'] + wed_tickets.loc[wed_tickets['PLZ']==1865,'ticket']
wed_tickets = wed_tickets[wed_tickets['PLZ'] != 1865]
thu_tickets.loc[thu_tickets['PLZ']==1854,'ticket']=thu_tickets.loc[thu_tickets['PLZ']==1854,'ticket'] + thu_tickets.loc[thu_tickets['PLZ']==1865,'ticket']
thu_tickets = thu_tickets[thu_tickets['PLZ'] != 1865]
fri_tickets.loc[fri_tickets['PLZ']==1854,'ticket']=fri_tickets.loc[fri_tickets['PLZ']==1854,'ticket'] + fri_tickets.loc[fri_tickets['PLZ']==1865,'ticket']
fri_tickets = fri_tickets[fri_tickets['PLZ'] != 1865]
sat_tickets.loc[sat_tickets['PLZ']==1854,'ticket']=sat_tickets.loc[sat_tickets['PLZ']==1854,'ticket'] + sat_tickets.loc[sat_tickets['PLZ']==1865,'ticket']
sat_tickets = sat_tickets[sat_tickets['PLZ'] != 1865]
sub_tickets.loc[sub_tickets['PLZ']==1854,'ticket']=sub_tickets.loc[sub_tickets['PLZ']==1854,'ticket'] + sub_tickets.loc[sub_tickets['PLZ']==1865,'ticket']
sub_tickets = sub_tickets[sub_tickets['PLZ'] != 1865]

In [51]:
print(wed_tickets[wed_tickets['Canton']=='VD']['ticket'].sum())
print(thu_tickets[thu_tickets['Canton']=='VD']['ticket'].sum())
print(fri_tickets[fri_tickets['Canton']=='VD']['ticket'].sum())
print(sat_tickets[sat_tickets['Canton']=='VD']['ticket'].sum())
print(sub_tickets[sub_tickets['Canton']=='VD']['ticket'].sum())
print(all_tickets[all_tickets['Canton']=='VD']['ticket'].sum())

8352.0
7268
4705.0
6021
290
26636.0


In [52]:
full_data = cities_to_serve.merge(wed_tickets[wed_tickets['Canton']=='VD'][['PLZ','ticket']], how='left', left_on='NPA', right_on='PLZ')
full_data.drop(columns=['PLZ'], inplace=True)
full_data.rename(columns={'ticket': 'wed_tickets'}, inplace=True)
full_data = full_data.merge(thu_tickets[thu_tickets['Canton']=='VD'][['PLZ','ticket']], how='left', left_on='NPA', right_on='PLZ')
full_data.drop(columns=['PLZ'], inplace=True)
full_data.rename(columns={'ticket': 'thu_tickets'}, inplace=True)
full_data = full_data.merge(fri_tickets[fri_tickets['Canton']=='VD'][['PLZ','ticket']], how='left', left_on='NPA', right_on='PLZ')
full_data.drop(columns=['PLZ'], inplace=True)
full_data.rename(columns={'ticket': 'fri_tickets'}, inplace=True)
full_data = full_data.merge(sat_tickets[sat_tickets['Canton']=='VD'][['PLZ','ticket']], how='left', left_on='NPA', right_on='PLZ')
full_data.drop(columns=['PLZ'], inplace=True)
full_data.rename(columns={'ticket': 'sat_tickets'}, inplace=True)
full_data = full_data.merge(sub_tickets[sub_tickets['Canton']=='VD'][['PLZ','ticket']], how='left', left_on='NPA', right_on='PLZ')
full_data.drop(columns=['PLZ'], inplace=True)
full_data.rename(columns={'ticket': 'sub_tickets'}, inplace=True)

In [53]:
full_data[['wed_tickets','thu_tickets','fri_tickets','sat_tickets','sub_tickets']].sum()

wed_tickets    8352.0
thu_tickets    7268.0
fri_tickets    4705.0
sat_tickets    6021.0
sub_tickets     290.0
dtype: float64

In [54]:
main_city0=npa_to_connect.iloc[0,0]
main_city1=npa_to_connect.iloc[0,1]
main_city2=npa_to_connect.iloc[0,2]
trainline0=npa_to_connect.iloc[1:,0]
trainline1=npa_to_connect.iloc[1:,1]
trainline2=npa_to_connect.iloc[1:,2]

In [55]:
trainline0.drop_duplicates(inplace=True)
trainline1.drop_duplicates(inplace=True)
trainline2.drop_duplicates(inplace=True)

In [56]:
aggregated_demand_line0=pd.DataFrame(full_data[full_data['NPA'].isin(trainline0)].sum())
aggregated_demand_line1=pd.DataFrame(full_data[full_data['NPA'].isin(trainline1)].sum())
aggregated_demand_line2=pd.DataFrame(full_data[full_data['NPA'].isin(trainline2)].sum())

In [57]:
full_data.loc[full_data['NPA'] == main_city0,'wed_tickets']=full_data.loc[full_data['NPA'] == main_city0,'wed_tickets'] + aggregated_demand_line0.loc['wed_tickets'][0]
full_data.loc[full_data['NPA'] == main_city0,'thu_tickets']=full_data.loc[full_data['NPA'] == main_city0,'thu_tickets']+aggregated_demand_line0.loc['thu_tickets'][0]
full_data.loc[full_data['NPA'] == main_city0,'fri_tickets']=full_data.loc[full_data['NPA'] == main_city0,'fri_tickets']+aggregated_demand_line0.loc['fri_tickets'][0]
full_data.loc[full_data['NPA'] == main_city0,'sat_tickets']=full_data.loc[full_data['NPA'] == main_city0,'sat_tickets']+aggregated_demand_line0.loc['sat_tickets'][0]
full_data.loc[full_data['NPA'] == main_city0,'sub_tickets']=full_data.loc[full_data['NPA'] == main_city0,'sub_tickets']+aggregated_demand_line0.loc['sub_tickets'][0]

full_data.loc[full_data['NPA'] == main_city1,'wed_tickets']=full_data.loc[full_data['NPA'] == main_city1,'wed_tickets'] + aggregated_demand_line1.loc['wed_tickets'][0]
full_data.loc[full_data['NPA'] == main_city1,'thu_tickets']=full_data.loc[full_data['NPA'] == main_city1,'thu_tickets']+aggregated_demand_line1.loc['thu_tickets'][0]
full_data.loc[full_data['NPA'] == main_city1,'fri_tickets']=full_data.loc[full_data['NPA'] == main_city1,'fri_tickets']+aggregated_demand_line1.loc['fri_tickets'][0]
full_data.loc[full_data['NPA'] == main_city1,'sat_tickets']=full_data.loc[full_data['NPA'] == main_city1,'sat_tickets']+aggregated_demand_line1.loc['sat_tickets'][0]
full_data.loc[full_data['NPA'] == main_city1,'sub_tickets']=full_data.loc[full_data['NPA'] == main_city1,'sub_tickets']+aggregated_demand_line1.loc['sub_tickets'][0]

full_data.loc[full_data['NPA'] == main_city2,'wed_tickets']=full_data.loc[full_data['NPA'] == main_city2,'wed_tickets'] + aggregated_demand_line2.loc['wed_tickets'][0]
full_data.loc[full_data['NPA'] == main_city2,'thu_tickets']=full_data.loc[full_data['NPA'] == main_city2,'thu_tickets']+aggregated_demand_line2.loc['thu_tickets'][0]
full_data.loc[full_data['NPA'] == main_city2,'fri_tickets']=full_data.loc[full_data['NPA'] == main_city2,'fri_tickets']+aggregated_demand_line2.loc['fri_tickets'][0]
full_data.loc[full_data['NPA'] == main_city2,'sat_tickets']=full_data.loc[full_data['NPA'] == main_city2,'sat_tickets']+aggregated_demand_line2.loc['sat_tickets'][0]
full_data.loc[full_data['NPA'] == main_city2,'sub_tickets']=full_data.loc[full_data['NPA'] == main_city2,'sub_tickets']+aggregated_demand_line2.loc['sub_tickets'][0]

In [58]:
full_data['NPA'].nunique()

309

In [59]:
full_data = full_data[~full_data['NPA'].isin(trainline0)]
full_data = full_data[~full_data['NPA'].isin(trainline1)]
full_data = full_data[~full_data['NPA'].isin(trainline2)]

In [60]:
full_data[['wed_tickets','thu_tickets','fri_tickets','sat_tickets','sub_tickets']].sum()

wed_tickets    8352.0
thu_tickets    7268.0
fri_tickets    4705.0
sat_tickets    6021.0
sub_tickets     286.0
dtype: float64

In [61]:
full_data[['wed_tickets','thu_tickets','fri_tickets','sat_tickets','sub_tickets']].sum(axis=0).sum()

26632.0

In [62]:
full_data['all_tickets'] = full_data[['wed_tickets','thu_tickets','fri_tickets','sat_tickets','sub_tickets']].sum(axis=1)
full_data

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,N_lv03,lat,lng,distance_to_venoge_km,wed_tickets,thu_tickets,fri_tickets,sat_tickets,sub_tickets,all_tickets
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,161298.778,46.599406,6.540556,0.74,161.0,181.0,166.0,203.0,59.0,770.0
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,162919.512,46.613915,6.531495,2.72,183.0,151.0,150.0,142.0,29.0,655.0
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,162344.417,46.608998,6.564397,3.51,32.0,17.0,14.0,29.0,1.0,93.0
4,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,164336.340,46.626796,6.548602,3.98,58.0,48.0,33.0,55.0,8.0,202.0
5,Sullens,1036,Sullens,5501,VD,2533207.010,1160372.401,fr,1225,12,...,160372.401,46.591278,6.567058,4.12,35.0,50.0,32.0,30.0,4.0,151.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,La Comballaz,1862,Ormont-Dessous,5410,VD,2572060.524,1136176.786,fr,1244,8,...,136176.786,46.376379,7.075504,72.24,NaN,4.0,NaN,NaN,NaN,4.0
305,Rougemont,1659,Château-d'Oex,5841,VD,2580115.427,1147183.777,fr,3656,36,...,147183.777,46.475680,7.179722,74.02,5.0,NaN,NaN,NaN,NaN,5.0
306,Leysin,1854,Leysin,5407,VD,2566911.342,1133415.264,fr,3685,23,...,133415.264,46.351303,7.008778,75.77,NaN,NaN,NaN,NaN,NaN,0.0
307,Gryon,1882,Gryon,5405,VD,2573628.043,1126382.007,fr,1525,10,...,126382.007,46.288333,7.096432,77.00,3.0,4.0,NaN,5.0,NaN,12.0


In [63]:
full_data=full_data[full_data['all_tickets']!=0]
full_data

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,N_lv03,lat,lng,distance_to_venoge_km,wed_tickets,thu_tickets,fri_tickets,sat_tickets,sub_tickets,all_tickets
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,161298.778,46.599406,6.540556,0.74,161.0,181.0,166.0,203.0,59.0,770.0
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,162919.512,46.613915,6.531495,2.72,183.0,151.0,150.0,142.0,29.0,655.0
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,162344.417,46.608998,6.564397,3.51,32.0,17.0,14.0,29.0,1.0,93.0
4,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,164336.340,46.626796,6.548602,3.98,58.0,48.0,33.0,55.0,8.0,202.0
5,Sullens,1036,Sullens,5501,VD,2533207.010,1160372.401,fr,1225,12,...,160372.401,46.591278,6.567058,4.12,35.0,50.0,32.0,30.0,4.0,151.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,La Forclaz VD,1866,Ollon,5409,VD,2569188.595,1131844.374,fr,8222,52,...,131844.374,46.337281,7.038465,72.05,1.0,NaN,NaN,NaN,NaN,1.0
304,La Comballaz,1862,Ormont-Dessous,5410,VD,2572060.524,1136176.786,fr,1244,8,...,136176.786,46.376379,7.075504,72.24,NaN,4.0,NaN,NaN,NaN,4.0
305,Rougemont,1659,Château-d'Oex,5841,VD,2580115.427,1147183.777,fr,3656,36,...,147183.777,46.475680,7.179722,74.02,5.0,NaN,NaN,NaN,NaN,5.0
307,Gryon,1882,Gryon,5405,VD,2573628.043,1126382.007,fr,1525,10,...,126382.007,46.288333,7.096432,77.00,3.0,4.0,NaN,5.0,NaN,12.0


In [64]:
wed_tickets_out_VD = wed_tickets[wed_tickets['Canton']!='VD']['ticket'].sum()
thu_tickets_out_VD = thu_tickets[thu_tickets['Canton']!='VD']['ticket'].sum()
fri_tickets_out_VD = thu_tickets[thu_tickets['Canton']!='VD']['ticket'].sum()
sat_tickets_out_VD = thu_tickets[thu_tickets['Canton']!='VD']['ticket'].sum()
sub_tickets_out_VD = sub_tickets[sub_tickets['Canton']!='VD']['ticket'].sum()


In [65]:
#Assign demand outside of Canton of Vaud to Cossonay
full_data.loc[full_data['NPA']==1304,'wed_tickets'] = full_data.loc[full_data['NPA']==1304,'wed_tickets'] + wed_tickets_out_VD
full_data.loc[full_data['NPA']==1304,'thu_tickets'] = full_data.loc[full_data['NPA']==1304,'thu_tickets'] + thu_tickets_out_VD
full_data.loc[full_data['NPA']==1304,'fri_tickets'] = full_data.loc[full_data['NPA']==1304,'fri_tickets'] + fri_tickets_out_VD
full_data.loc[full_data['NPA']==1304,'sat_tickets'] = full_data.loc[full_data['NPA']==1304,'sat_tickets'] + sat_tickets_out_VD
full_data.loc[full_data['NPA']==1304,'sub_tickets'] = full_data.loc[full_data['NPA']==1304,'sub_tickets'] + sub_tickets_out_VD



In [66]:
full_data[['wed_tickets','thu_tickets','fri_tickets','sat_tickets','sub_tickets']].sum()

wed_tickets    12959.0
thu_tickets     9066.0
fri_tickets     6503.0
sat_tickets     7819.0
sub_tickets      319.0
dtype: float64

In [67]:
full_data[['wed_tickets','thu_tickets','fri_tickets','sat_tickets','sub_tickets']].sum(axis=0).sum()

36666.0

In [68]:
full_data['sub_tickets'] = full_data['sub_tickets'].fillna(0)

/var/folders/3d/zlkn8h8d2yx5345bf7csn9bh0000gn/T/ipykernel_17158/3350464156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data['sub_tickets'] = full_data['sub_tickets'].fillna(0)


In [69]:
#Add the subscription tickets to everyday, as we assume they come to the festival everyday
full_data['wed_tickets'] = full_data['wed_tickets'] + full_data['sub_tickets']
full_data['thu_tickets'] = full_data['thu_tickets'] + full_data['sub_tickets']
full_data['fri_tickets'] = full_data['fri_tickets'] + full_data['sub_tickets']
full_data['sat_tickets'] = full_data['sat_tickets'] + full_data['sub_tickets']

/var/folders/3d/zlkn8h8d2yx5345bf7csn9bh0000gn/T/ipykernel_17158/2725305968.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data['wed_tickets'] = full_data['wed_tickets'] + full_data['sub_tickets']
/var/folders/3d/zlkn8h8d2yx5345bf7csn9bh0000gn/T/ipykernel_17158/2725305968.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data['thu_tickets'] = full_data['thu_tickets'] + full_data['sub_tickets']
/var/folders/3d/zlkn8h8d2yx5345bf7csn9bh0000gn/T/ipykernel_17158/2725305968.py:4: SettingWithCopyW

In [70]:
full_data

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,N_lv03,lat,lng,distance_to_venoge_km,wed_tickets,thu_tickets,fri_tickets,sat_tickets,sub_tickets,all_tickets
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,161298.778,46.599406,6.540556,0.74,220.0,240.0,225.0,262.0,59.0,770.0
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,162919.512,46.613915,6.531495,2.72,212.0,180.0,179.0,171.0,29.0,655.0
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,162344.417,46.608998,6.564397,3.51,33.0,18.0,15.0,30.0,1.0,93.0
4,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,164336.340,46.626796,6.548602,3.98,66.0,56.0,41.0,63.0,8.0,202.0
5,Sullens,1036,Sullens,5501,VD,2533207.010,1160372.401,fr,1225,12,...,160372.401,46.591278,6.567058,4.12,39.0,54.0,36.0,34.0,4.0,151.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,La Forclaz VD,1866,Ollon,5409,VD,2569188.595,1131844.374,fr,8222,52,...,131844.374,46.337281,7.038465,72.05,1.0,NaN,NaN,NaN,0.0,1.0
304,La Comballaz,1862,Ormont-Dessous,5410,VD,2572060.524,1136176.786,fr,1244,8,...,136176.786,46.376379,7.075504,72.24,NaN,4.0,NaN,NaN,0.0,4.0
305,Rougemont,1659,Château-d'Oex,5841,VD,2580115.427,1147183.777,fr,3656,36,...,147183.777,46.475680,7.179722,74.02,5.0,NaN,NaN,NaN,0.0,5.0
307,Gryon,1882,Gryon,5405,VD,2573628.043,1126382.007,fr,1525,10,...,126382.007,46.288333,7.096432,77.00,3.0,4.0,NaN,5.0,0.0,12.0


In [71]:
full_data[['Commune','NPA']].to_csv('cities_to_serve.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

# Computing demand matrix

In [72]:
full_data.to_csv('full_data.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
full_data['wed_tickets'].to_csv('demand_wed.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)
full_data['thu_tickets'].to_csv('demand_thu.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)
full_data['fri_tickets'].to_csv('demand_fri.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)
full_data['sat_tickets'].to_csv('demand_sat.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

# Computing distance matrix

In [ ]:
full_data.reset_index(drop=True, inplace=True)
full_data

In [ ]:
n=full_data.shape[0]
n

In [ ]:
coordinates = full_data.apply(lambda row: [row['lng'], row['lat']], axis=1)


In [ ]:
#Initialize empty matrix
distance_matrix = [[None] * n for _ in range(n)]

# Save the empty matrix to a CSV file
with open('distance_matrix.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(distance_matrix)

In [ ]:
# Initialize a cache to store distances
distance_cache = {}

# Load the distance matrix from the CSV file
with open('distance_matrix.csv', 'r') as f:
    reader = csv.reader(f)
    distance_matrix = [[float(cell) if cell else None for cell in row] for row in reader]


In [ ]:

# Fill the matrix row by row
for i in range(n):
    for j in range(i, n):  # Only calculate for j >= i
        if distance_matrix[i][j] is None:
            if i == j:
                distance_matrix[i][j] = float('inf')
            else:
                # Check if the distance is already in the cache
                coord_pair = tuple(sorted([tuple(coordinates[i]), tuple(coordinates[j])]))
                if coord_pair in distance_cache:
                    distance_km = distance_cache[coord_pair]
                else:
                    try:
                        response = client.directions(
                            profile='driving-car',
                            format='geojson',
                            coordinates=[coordinates[i], coordinates[j]]
                        )
                        distance_km = response['features'][0]['properties']['segments'][0]['distance'] / 1000.0
                        distance_cache[coord_pair] = round(distance_km)  # Cache the result
                    except ApiError as e:
                        print(f"API error from point {i} to {j}: {e}")
                        distance_km = None
                    except Exception as e:
                        print(f"Unexpected error from point {i} to {j}: {e}")
                        distance_km = None

                distance_matrix[i][j] = distance_km
                distance_matrix[j][i] = distance_km  # Reuse for the symmetric pair
                

In [ ]:
# Display the distance matrix
for row in distance_matrix:
    print(row)

In [ ]:
# Save the distance matrix to a CSV file
with open('distance_matrix.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(distance_matrix)